# Разработка сентимент-анализа под задачу заказчика
К вашей компании пришел заказчик, которому нужно решение задачи анализа тональности отзывов на товары. Заказчик хочет, чтобы вы оценили возможное качество работы такого алгоритма на небольшой тестовой выборке. При этом больше никаких данных вам не предоставляется. Требуется, чтобы качество работы вашего алгоритма (по accuracy) было строго больше 85%.
Оценка качества в этом задании реализована через контест на Kaggle Inclass:
https://inclass.kaggle.com/c/product-reviews-sentiment-analysis

## Парсинг
Отзывы и оценки будем парсить с сайта https://torg.mail.ru

In [2]:
# coding: utf8
import requests
from bs4 import BeautifulSoup
import csv
from time import sleep
from random import randint

data = []

def get_html(url):
    r = requests.get(url)
    return r.text

def write_csv(data):
    with open('reviews_train.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow((data[0], data[1]))

def get_page_data(html):
    soup = BeautifulSoup(html, 'lxml')
    ads = soup.find_all('div', class_='review-item__body')
    for ad in ads:
        try:
            review = ad.find('div', class_='review-item__content').text.strip()
        except:
            review = ''
        try:
            rating = ad.find('span', class_='review-item__rating-counter').text.strip()
        except:
            rating = ''

        data = [review.encode('utf8'), rating]
        write_csv(data)
      

base_url = 'https://torg.mail.ru/review/goods/mobilephones/?page='


for i in range(1, 881):
    url_gen = base_url + str(i)
    html = get_html(url_gen)
    get_page_data(html)

# Чтобы не забанили используем случайную задержку между скачиванием страниц
    sleep(randint(1,3))

В результате получаем файл reviews_train.csv, содержащий отзывы на телефоны и оценки, выставленные покупателями от 0 до 5

## Загружаем отзывы

In [67]:
import pandas as pd

data = pd.read_csv('reviews_train.csv',',', header=None)
data.columns = ['text','rating']

In [68]:
data.rating.value_counts()

5      7701
4,5    3690
4      2422
3      1179
3,5     922
1       898
2       350
2,5     340
1,5      94
0         4
Name: rating, dtype: int64

Будем считать негативными отзывы от 1 до 3, позитывными отзывы с 5 баллами.

In [69]:
data['res'] = 'NaN'
for i in range(len(data)):
    if data['rating'][i] == '1' or data['rating'][i] == '1,5' or data['rating'][i] == '2' or data['rating'][i] == '2,5' or data['rating'][i] == '3': 
        data['res'][i] = 0
    else:
        data['res'][i] = 'NaN'
    if data['rating'][i] == '5': 
        data['res'][i] = 1  

In [70]:
data.res.value_counts()

1      7701
NaN    7038
0      2861
Name: res, dtype: int64

In [71]:
data1 = data[data['res']==1]
data0 = data[data['res']==0]
data3 = pd.concat([data0, data1])

In [72]:
text_utf8 = data3['text']
res = list(data3['res'])

In [73]:
# Преобразуем из utf8 в unicode
text_unicode = []
for row in text_utf8:
    text_unicode.append(row.decode('utf8'))

In [74]:
# Чистим текст от небуквенных символов
import re

textr=[]
for row in text_unicode:
    textr.append(re.sub('[\n\t\r\s]',' ', row))

In [75]:
# Чистим от лишних пробелов
text = []
for row in textr:
    text.append(re.sub(r'\s+', ' ', row))

In [77]:
# Посмотрим что получилось
print text[29]

Компания сделала шаг назад выпустив такуюмодель. Следующий смартфон у меня будет явно не ASUS Достоинства Большая батарея, 4G, wi-fi, android 7. Недостатки Плохое стекло, большой экран,один слот под симку или карту памяти, нест стабилизации при фотосъемке, качество фото среднее.


## Выбираем модель

Составим baseline из CountVectorizer, TfidfTransformer и классификаторов, встречавшихся ранее в курсе с параметрами по умолчанию. Посчитаем accuracy и выберем модели для дальнейшей оптимизации.

In [42]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV

In [43]:
rs = 2

parameters = {'classifier': [
    LogisticRegression(random_state=rs), 
    LinearSVC(random_state=rs), 
    SGDClassifier(random_state=rs),  
    SVC(random_state=rs),
    MultinomialNB(),   
    ]}

pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('transformer', TfidfTransformer()),
    ('classifier',  LogisticRegression(random_state=rs))
    ])

grid_search = GridSearchCV(pipeline, parameters, cv=5, scoring='accuracy', verbose=True, n_jobs=5)
grid_search.fit(text, res)
grid_search.grid_scores_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=5)]: Done  25 out of  25 | elapsed:  1.8min finished
/Users/rusa/anaconda/lib/python2.7/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.82002, std: 0.03066, params: {'classifier': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)},
 mean: 0.82323, std: 0.03691, params: {'classifier': LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=2, tol=0.0001,
     verbose=0)},
 mean: 0.82873, std: 0.03646, params: {'classifier': SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=2, shuffle=True, verbose=0,
       warm_start=False)},
 mean: 0.72912, std: 0.00009, params: {'classifier': SVC(C=1.0, cach

Выбираем те, у которых accuracy > 0.82, это LogisticRegression, LinearSVC, SGDClassifier. Далее будем подбирать для них параметры.
Внимание! Считается очень долго!

##  LogisticRegression

In [44]:
parameters = {
               'vectorizer__ngram_range': [(1, 2), (1, 3)],
               'vectorizer__max_df': [0.60, 0.65, 0.70, 0.75],
    
               'transformer__use_idf': [False, True],
               'transformer__smooth_idf': [False, True],
               'transformer__sublinear_tf': [False, True],
                                  
               'classifier__max_iter': [5, 10, 11, 12, 13, 15],
               'classifier__C': [1.0, 3.0, 5.0, 10.0, 15.0, 20.0]
              }

pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('transformer', TfidfTransformer()),
    ('classifier',  LogisticRegression(random_state=rs))
    ])
grid_search = GridSearchCV(pipeline, parameters, cv=5, scoring='accuracy', verbose=True, n_jobs=5)
grid_search.fit(text, res)
print grid_search.best_score_
print grid_search.best_params_

Fitting 5 folds for each of 2304 candidates, totalling 11520 fits


[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  2.4min
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed: 11.7min
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed: 27.1min
[Parallel(n_jobs=5)]: Done 790 tasks      | elapsed: 49.5min
[Parallel(n_jobs=5)]: Done 1240 tasks      | elapsed: 77.8min
[Parallel(n_jobs=5)]: Done 1790 tasks      | elapsed: 117.0min
[Parallel(n_jobs=5)]: Done 2440 tasks      | elapsed: 157.6min
[Parallel(n_jobs=5)]: Done 3190 tasks      | elapsed: 206.7min
[Parallel(n_jobs=5)]: Done 4040 tasks      | elapsed: 261.3min
[Parallel(n_jobs=5)]: Done 4990 tasks      | elapsed: 322.2min
[Parallel(n_jobs=5)]: Done 6040 tasks      | elapsed: 388.9min
[Parallel(n_jobs=5)]: Done 7190 tasks      | elapsed: 464.2min
[Parallel(n_jobs=5)]: Done 8440 tasks      | elapsed: 544.4min
[Parallel(n_jobs=5)]: Done 9790 tasks      | elapsed: 632.8min
[Parallel(n_jobs=5)]: Done 11240 tasks      | elapsed: 728.0min
[Parallel(n_jobs=5)]: Done 11520 out of 11520 | elapsed: 746.6m

0.844442340466
{'vectorizer__ngram_range': (1, 2), 'classifier__max_iter': 10, 'vectorizer__max_df': 0.75, 'transformer__sublinear_tf': True, 'transformer__smooth_idf': True, 'transformer__use_idf': True, 'classifier__C': 15.0}


## LinearSVC

In [45]:
parameters = {
               'vectorizer__ngram_range': [(1, 2), (1, 3)],
               'vectorizer__max_df': [0.60, 0.65, 0.70, 0.75],
               
               'transformer__use_idf': [False, True],
               'transformer__smooth_idf': [False, True],
               'transformer__sublinear_tf': [False, True],
                         
               'classifier__C' : [1.0, 3.0, 5.0]
              }

pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('transformer', TfidfTransformer()),
    ('classifier',  LinearSVC(random_state=rs))
    ])
grid_search = GridSearchCV(pipeline, parameters, cv=5, scoring='accuracy', verbose=True, n_jobs=5)
grid_search.fit(text, res)
print grid_search.best_score_
print grid_search.best_params_

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  2.4min
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed: 11.4min
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed: 27.1min
[Parallel(n_jobs=5)]: Done 790 tasks      | elapsed: 49.8min
[Parallel(n_jobs=5)]: Done 960 out of 960 | elapsed: 61.6min finished


0.847945464874
{'vectorizer__ngram_range': (1, 3), 'vectorizer__max_df': 0.75, 'transformer__sublinear_tf': True, 'transformer__smooth_idf': True, 'transformer__use_idf': True, 'classifier__C': 5.0}


## SGDClassifier

In [47]:
parameters = {
               'vectorizer__max_features': [None],
               'vectorizer__ngram_range': [(1, 2), (1, 3)],
               'vectorizer__max_df': [0.60, 0.65, 0.70, 0.75],
    
               'transformer__use_idf': [False, True],
               'transformer__smooth_idf': [False, True],
               'transformer__sublinear_tf': [False, True],
            
               'classifier__alpha': [1e-4, 1e-5],
               'classifier__n_iter': [5, 50, 100, 500, 1000]
              }

pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('transformer', TfidfTransformer()),
    ('classifier',  SGDClassifier(random_state=rs))
    ])
grid_search = GridSearchCV(pipeline, parameters, cv=5, scoring='accuracy', verbose=True, n_jobs=5)
grid_search.fit(text, res)
print grid_search.best_score_
print grid_search.best_params_

Fitting 5 folds for each of 640 candidates, totalling 3200 fits


[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  2.5min
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed: 12.8min
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed: 28.1min
[Parallel(n_jobs=5)]: Done 790 tasks      | elapsed: 53.3min
[Parallel(n_jobs=5)]: Done 1240 tasks      | elapsed: 92.8min
[Parallel(n_jobs=5)]: Done 1790 tasks      | elapsed: 158.8min
[Parallel(n_jobs=5)]: Done 2440 tasks      | elapsed: 204.5min
[Parallel(n_jobs=5)]: Done 3190 tasks      | elapsed: 294.3min
[Parallel(n_jobs=5)]: Done 3200 out of 3200 | elapsed: 295.7min finished


0.850123082749
{'vectorizer__ngram_range': (1, 2), 'classifier__alpha': 0.0001, 'vectorizer__max_features': None, 'vectorizer__max_df': 0.75, 'classifier__n_iter': 100, 'transformer__sublinear_tf': False, 'transformer__smooth_idf': False, 'transformer__use_idf': True}


## Бинго!

Наилучший результат 0.85 показал алгоритм с SGDClassifier.

## Учим модель

In [49]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1,2), max_df=0.75)),
    ('transformer', TfidfTransformer(use_idf=True, smooth_idf=False, sublinear_tf=False)),
    ('classifier',  SGDClassifier(random_state=rs, alpha=1e-4, n_iter=100,))
    ])
pipeline.fit(text,res)
pipeline

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.75, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
   ...      penalty='l2', power_t=0.5, random_state=2, shuffle=True, verbose=0,
       warm_start=False))])

## Загружаем test отзывы

In [78]:
from lxml import etree

f = open("sss.xml",'w')
f.write('<root>\n')
f2 = open('test.csv','r')
lines = f2.readlines()

for line in lines: 
    f.write(line)
f2.close()
f.write('</root>')
f.close()

f = open('sss.xml', 'r')
xml = etree.parse(f)
test = xml.findall('review')
test_unicode = [t.text for t in test]

In [79]:
# Чистим текст от небуквенных символов
import re

testr=[]
for row in test_unicode:
    testr.append(re.sub('[\n\t\r\s]',' ', row))

In [80]:
# Чистим от лишних пробелов
test=[]
for row in testr:
    test.append(re.sub(r'\s+', ' ', row))

In [81]:
print test[10]

Метттлленнныййй-ммеееттлленный. Ну что это такое, невозможно просто выключить калькулятор, тел. спрашивает меня, уверена ли я, закрыть ли приложение? А потом долго показывает кружочек, закрываю-де, сейчас-сейчас! Долго включается-загружается. Это что у меня, перегруженный смартфон с десятью фильмами на борту, или элементарный простой телефон? Меньше года в работе, стал на исходящие вызовы показывать мне "вызов запрещен", лечится перезагрузкой. КЕМ запрещен??????? Игр не стоит, фоток минимум, в интернет не ходила. Очень тупой, неудобный, я НЕ довольна, почти за год так и не привыкла, только стал больше раздражать. Набирать номер не удобно. После перезагрузки ненормально долго "список контактов недоступен", не позвонить. Кажется, что прием сигнала чаще, чем нужно, падает. Оставлю отзыв, поскольку телефон еще продается. 


## Получаем предсказания

In [90]:
result = pd.DataFrame(pipeline.predict(test), columns=['y1'])
result.index.name='Id'

In [91]:
# Преобразуем к тербуемому формату ответа

result['y'] = 'neg'
result['y'][result['y1']==1]='pos'
result = result.drop(['y1'], axis=1)

/Users/rusa/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [66]:
result1.to_csv('result.csv', sep=',', header=True, index=True)
!cat result.csv

Id,y
0,neg
1,pos
2,neg
3,neg
4,pos
5,pos
6,pos
7,pos
8,neg
9,pos
10,neg
11,pos
12,pos
13,pos
14,pos
15,pos
16,pos
17,pos
18,pos
19,pos
20,pos
21,neg
22,neg
23,pos
24,pos
25,neg
26,pos
27,pos
28,neg
29,pos
30,neg
31,pos
32,neg
33,pos
34,neg
35,pos
36,pos
37,pos
38,pos
39,pos
40,pos
41,pos
42,pos
43,neg
44,neg
45,pos
46,pos
47,pos
48,neg
49,neg
50,neg
51,neg
52,neg
53,neg
54,neg
55,pos
56,pos
57,pos
58,pos
59,pos
60,neg
61,pos
62,neg
63,neg
64,neg
65,neg
66,neg
67,neg
68,pos
69,pos
70,pos
71,neg
72,neg
73,neg
74,pos
75,pos
76,neg
77,pos
78,neg
79,neg
80,neg
81,pos
82,neg
83,pos
84,pos
85,pos
86,pos
87,pos
88,neg
89,neg
90,pos
91,pos
92,neg
93,pos
94,neg
95,pos
96,pos
97,neg
98,pos
99,pos


# Загружаем на https://inclass.kaggle.com/c/product-reviews-sentiment-analysis   Результат 0.90